Start with some imports

In [1]:
import pandapower as pp
import pandapower.networks as nw
import pandapower.plotting as pplt
import matplotlib.pyplot as plt
import numpy as np
import numba
import pandas as pd
import pulp as pl
import xlsxwriter as xl

Create the CIGRE network

In [2]:

net = nw.create_cigre_network_mv()

1.1 State of the network

In [3]:
pp.runpp(net)
print("Loads")
print(net.res_load.p_mw)
print("Bus voltages")
print(net.res_bus.vm_pu)
print("Trafo loading")
print(net.res_trafo.loading_percent)
print("Line loading")
net.res_line.loading_percent

Loads
0     14.99400
1      0.27645
2      0.43165
3      0.72750
4      0.54805
5      0.58685
6      0.47530
7      0.32980
8     14.99400
9      0.20855
10     4.84500
11     0.22525
12     0.07650
13     0.57375
14     0.06800
15     5.01600
16     0.03400
17     0.33150
Name: p_mw, dtype: float64
Bus voltages
0     1.030000
1     0.991972
2     0.968147
3     0.930961
4     0.929098
5     0.927823
6     0.926321
7     0.925122
8     0.925404
9     0.924422
10    0.923174
11    0.922980
12    1.000146
13    0.995326
14    0.992553
Name: vm_pu, dtype: float64
Trafo loading
0    101.411473
1     84.698048
Name: loading_percent, dtype: float64
Line loading


0     96.482978
1     96.958807
2     37.580668
3     28.091021
4     12.122901
5      1.936801
6     33.550914
7     19.476278
8      7.294089
9     48.143637
10     9.476242
11     8.908939
12     0.084082
13     0.171049
14     0.037287
Name: loading_percent, dtype: float64

Check initial switch positions

In [15]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Load R1,1,14.99400,3.044662,0.0,0.0,15.300,1.0,True,wye
1,Load R3,3,0.27645,0.069285,0.0,0.0,0.285,1.0,True,wye
2,Load R4,4,0.43165,0.108182,0.0,0.0,0.445,1.0,True,wye
3,Load R5,5,0.72750,0.182329,0.0,0.0,0.750,1.0,True,wye
4,Load R6,6,0.54805,0.137354,0.0,0.0,0.565,1.0,True,wye
5,Load R8,8,0.58685,0.147078,0.0,0.0,0.605,1.0,True,wye
6,Load R10,10,0.47530,0.119121,0.0,0.0,0.490,1.0,True,wye
7,Load R11,11,0.32980,0.082656,0.0,0.0,0.340,1.0,True,wye
8,Load R12,12,14.99400,3.044662,0.0,0.0,15.300,1.0,True,wye
9,Load R14,14,0.20855,0.052268,0.0,0.0,0.215,1.0,True,wye


1.2 N-1 analysis

In [ ]:
# limits
vmax = 1.1
vmin = .9
max_ll = 100.

lines = net.line.index
critical = list()

for l in lines:
    net.line.loc[l, "in_service"] = False
    pp.runpp(net)

    if net.res_bus.vm_pu.max() > vmax or net.res_bus.vm_pu.min() < vmin or net.res_line.loading_percent.max() > max_ll:
        critical.append(l)
    net.line.loc[l, "in_service"] = True

ax = pplt.simple_plot(net, show_plot=False)
clc = pplt.create_line_collection(net, critical, color="r", linewidth=3., use_bus_geodata=True)
pplt.draw_collections([clc], ax=ax)
plt.show()

1.3 N-1 with smaller limits

In [ ]:
# limits
vmax = 1.05
vmin = .95
max_ll = 100.

lines = net.line.index
critical = list()

for l in lines:
    net.line.loc[l, "in_service"] = False
    pp.runpp(net)

    if net.res_bus.vm_pu.max() > vmax or net.res_bus.vm_pu.min() < vmin or net.res_line.loading_percent.max() > max_ll:
        critical.append(l)
    net.line.loc[l, "in_service"] = True

ax = pplt.simple_plot(net, show_plot=False)
clc = pplt.create_line_collection(net, critical, color="r", linewidth=3., use_bus_geodata=True)
pplt.draw_collections([clc], ax=ax)
plt.show()